In [1]:
import re
import string
import numpy as np
import random
import os

# os.environ['TF_NUM_INTEROP_THREADS'] = '4'
# os.environ['TF_NUM_INTRAOP_THREADS'] = '4'

import tensorflow as tf

In [2]:
with open('dict.txt') as f:
    base1984 = f.read()

In [3]:
print(base1984[:1000])

2
1080
&c
10-point
10th
11-point
12-point
16-point
18-point
1st
2,4,5-t
2,4-d
20-point
2D
2nd
30-30
3D
3-D
3M
3rd
48-point
4-D
4GL
4H
4th
5-point
5-T
5th
6-point
6th
7-point
7th
8-point
8th
9-point
9th
a
a'
a-
A&M
A&P
A.
A.A.A.
A.B.
A.B.A.
A.C.
A.D.
A.D.C.
A.F.
A.F.A.M.
A.G.
A.H.
A.I.
A.I.A.
A.I.D.
A.L.
A.L.P.
A.M.
A.M.A.
A.M.D.G.
A.N.
a.p.
a.r.
A.R.C.S.
A.U.
A.U.C.
A.V.
a.w.
A.W.O.L.
A/C
A/F
A/O
A/P
A/V
A1
A-1
A4
A5
AA
AAA
AAAA
AAAAAA
AAAL
AAAS
Aaberg
Aachen
AAE
AAEE
AAF
AAG
aah
aahed
aahing
aahs
AAII
aal
Aalborg
Aalesund
aalii
aaliis
aals
Aalst
Aalto
AAM
AAMSI
Aandahl
A-and-R
Aani
AAO
AAP
AAPSS
Aaqbiye
Aar
Aara
Aarau
AARC
aardvark
aardvarks
aardwolf
aardwolves
Aaren
Aargau
aargh
Aarhus
Aarika
Aaron
Aaronic
Aaronical
Aaronite
Aaronitic
Aaron's-beard
Aaronsburg
Aaronson
AARP
aarrgh
aarrghh
Aaru
AAS
A'asia
aasvogel
aasvogels
AAU
AAUP
AAUW
AAVSO
AAX
A-axes
A-axis
AB
ab-
ABA
Ababa
Ababdeh
Ababua
abac
abaca
abacay
abacas
abacate
abacaxi
abaci
abacinate
abacination
abacisci
abaciscus
abacis

In [4]:
# print("^[{}\s]+$".format(re.escape(''.join(filter(lambda x: x != "'", string.punctuation)))))
filterd1984 = base1984.split('\n')
# isPunct = re.compile("^[{}\s]+$".format(re.escape(string.punctuation)))
isPunctNoAp = re.compile("[{}§—„]+".format(re.escape(''.join(filter(lambda x: x != "'" and x != '.', string.punctuation)))))
filterd1984 = list(map(lambda s: ' '.join(filter(lambda x: not re.match(r'^\s*$', x) and not len(x) == 0,
    re.sub(r'\s+', ' ',
    isPunctNoAp.sub(' ',
    re.sub(r'(?<!\b)\'\b|\b\'(?!\b)', ' ',
    re.sub("-\s*\n", "",
    s.lower())))).split())), filterd1984))
print(filterd1984[:5])

['2', '1080', 'c', '10 point', '10th']


In [21]:
print("\n".join(filterd1984[:100]))

2
1080
c
10 point
10th
11 point
12 point
16 point
18 point
1st
2 4 5 t
2 4 d
20 point
2d
2nd
30 30
3d
3 d
3m
3rd
48 point
4 d
4gl
4h
4th
5 point
5 t
5th
6 point
6th
7 point
7th
8 point
8th
9 point
9th
a
a
a
a m
a p
a.
a.a.a.
a.b.
a.b.a.
a.c.
a.d.
a.d.c.
a.f.
a.f.a.m.
a.g.
a.h.
a.i.
a.i.a.
a.i.d.
a.l.
a.l.p.
a.m.
a.m.a.
a.m.d.g.
a.n.
a.p.
a.r.
a.r.c.s.
a.u.
a.u.c.
a.v.
a.w.
a.w.o.l.
a c
a f
a o
a p
a v
a1
a 1
a4
a5
aa
aaa
aaaa
aaaaaa
aaal
aaas
aaberg
aachen
aae
aaee
aaf
aag
aah
aahed
aahing
aahs
aaii
aal
aalborg
aalesund
aalii
aaliis


In [6]:
n = 16
mark = {}
for sent in filterd1984:
    for i in range(0, len(sent)+1):
        prev = sent[max(0, i-n):i]
        tt = mark.get(prev, '')
        if i == len(sent):
            mark[prev] = tt + '!'
        else:
            mark[prev] = tt + sent[i]

In [7]:
def finishSentMark(s):
    # assert len(s) >= n
    while True:
        crand = mark.get(s[-n:], '!')
        c = random.choice(crand)
        if c == '!':
            return s
        s += c

In [8]:
print(finishSentMark(''))

tasser


## "expected":

> The cops were armed 
with shotguns and tied Frank, who almost slipped on the 
ground, and also the other inmate, with an additional seat 
belt

## predicted:

> with shotguns and the policeman would finally you surrendered but he could not hold his stool out at arm's length of the dead seemed to strike down large crowds

## tailed questions

> for the moment is merely a piece of evidence of your sister isn't it

In [9]:
print(finishSentMark(''))

immuno


In [10]:
letters = '!' + ''.join(sorted(set(''.join(filterd1984))))
print(letters)
print(len(letters))

! '.0123456789abcdefghijklmnopqrstuvwxyz
40


In [11]:
from functools import reduce

"""
xy = \
    [ ( w[max(0, i-n):i], (w)[i:] ) # + '!'
    for w in filterd1984
    for i in range(len(w)+1)
    ]
"""

desiredlen = 100

# filterd1984_1 = list(filter(lambda x: len(x) <= desiredlen, filterd1984))
filterd1984_1 = list(map(lambda x: re.sub('\s+', '', x[:desiredlen]), filterd1984))
print(len(filterd1984_1))

maxlen = reduce(max, map(len, filterd1984_1))

xy = \
    [ ( w[:-1], w[1:] )
    for w in filterd1984_1
    ]

466551


In [12]:
xy[300]

('abattise', 'battises')

In [13]:
def recode(x):
    ggg = np.array([letters.index(c) for c in x])
    return np.append(ggg, np.zeros(maxlen - len(ggg)))

def recodey(y):
    ggg = np.array([letters.index(c) for c in y])
    a =  np.append(ggg, np.zeros(maxlen - len(ggg), dtype=int))
    b = np.zeros((a.size, len(letters)), dtype=int)
    try:
        b[np.arange(a.size, dtype=int), a] = 1
    except Exception as e:
        return b
    return b

xy_train = xy

xy1_x = np.array(
    [ recode(x).reshape(maxlen,1)
    for x, y in xy_train
    ])

xy1_y = np.array(
    [ recodey(y) #.reshape(maxlen, len(letters))
    for x, y in xy_train
    ])

In [14]:
print(xy1_y[0])

[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]


In [15]:
lstm = tf.keras.layers.LSTM(len(letters), return_sequences=True)
model = tf.keras.models.Sequential(
    [ lstm
    , tf.keras.layers.Dense(len(letters), activation='softmax')
    ])
print("x\t", xy1_x[0].shape)
print('out\t', lstm(xy1_x[0].reshape(1,*xy1_x[0].shape)).shape)
print("y\t", xy1_y[0].shape)
print("len\t", len(xy1_x))
print("sentlen\t", maxlen)

x	 (45, 1)
out	 (1, 45, 40)
y	 (45, 40)
len	 466551
sentlen	 45


In [16]:
# print(model.predict(xy1_x[0].reshape(1,*xy1_x[0].shape)))
model.compile(loss=tf.keras.losses.CategoricalCrossentropy(), 
              optimizer='sgd', # adagrad
              metrics=['accuracy'])
model.fit(xy1_x, xy1_y, epochs=3);
# print(model.predict(xy1_x[0].reshape(1,*xy1_x[0].shape)))

Epoch 1/3
14580/14580 [==============================] - 252s 17ms/step - loss: 0.5788 - accuracy: 0.8355
Epoch 2/3
14580/14580 [==============================] - 255s 17ms/step - loss: 0.5343 - accuracy: 0.8376
Epoch 3/3
14580/14580 [==============================] - 251s 17ms/step - loss: 0.5309 - accuracy: 0.8380


In [17]:
xy1_y

array([[[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]],

       [[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [1, 0, 0, ..., 0, 0, 0],
        [1, 0, 0, ..., 0, 0, 0],
        [1, 0, 0, ..., 0, 0, 0]],

       [[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]],

       ...,

       [[0, 0, 0, ..., 0, 0, 1],
        [0, 0, 0, ..., 0, 0, 0],
        [1, 0, 0, ..., 0, 0, 0],
        ...,
        [1, 0, 0, ..., 0, 0, 0],
        [1, 0, 0, ..., 0, 0, 0],
        [1, 0, 0, ..., 0, 0, 0]],

       [[0, 0, 0, ..., 0, 0, 1],
        [0, 0, 0, ..., 0, 0, 1],
        [1, 0, 0, ..., 0, 0, 0],
        ...,
        [1, 0, 0, ..., 

In [18]:
def runLSTM(x):
    print('from ', x)
    print(len(x))
    if len(x) > 0 and x[-1] == '!' or len(x) >= maxlen:
        return x
    x0 = x
    x = recode(x)
    x = x.reshape(1,*x.shape)
    p = model.predict(x)
    # print(p)
    p = np.argmax(p, axis=2)[0]
    print('#', ''.join(map(lambda x: letters[x], p)), sep='')
    p = p[len(x0)-1:]
    # p = np.round(p)
    # return x0 + ''.join(map(lambda x: letters[x], p))
    return runLSTM(x0 + letters[p[0]])
runLSTM('e')

from  e
1
#e!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
from  ee
2
#ee!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
from  eee
3
#eer!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
from  eeer
4
#eere!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
from  eeere
5
#eeree!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
from  eeeree
6
#eereee!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
from  eeereee
7
#eereeee!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
from  eeereeee
8
#eereeeee!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
from  eeereeeee
9
#eereeeeee!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
from  eeereeeeee
10
#eereeeeeee!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
from  eeereeeeeee
11
#eereeeeeeee!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
from  eeereeeeeeee
12
#eereeeeeeees!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
from  eeereeeeeeees
13
#eereeeeeeeese!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
from  eeereeeeeeeese
14
#eereeeeeeeeses!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
from  eeereeeeeeeeses
15
#eereeeeeeeesese!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
from  eeereeeeeeeesese
16
#eereeee

'eeereeeeeeeeseseseseseseseseseseseseseseseses'

In [19]:
runLSTM('q')

from  q
1
#e!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
from  qe
2
#ee!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
from  qee
3
#eer!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
from  qeer
4
#eere!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
from  qeere
5
#eeree!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
from  qeeree
6
#eereee!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
from  qeereee
7
#eereeee!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
from  qeereeee
8
#eereeeee!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
from  qeereeeee
9
#eereeeeee!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
from  qeereeeeee
10
#eereeeeeee!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
from  qeereeeeeee
11
#eereeeeeeee!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
from  qeereeeeeeee
12
#eereeeeeeees!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
from  qeereeeeeeees
13
#eereeeeeeeese!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
from  qeereeeeeeeese
14
#eereeeeeeeesee!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
from  qeereeeeeeeesee
15
#eereeeeeeeesees!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
from  qeereeeeeeeesees
16
#eereeee

'qeereeeeeeeeseesesesesesesesesesesesesesesese'

In [22]:
runLSTM('brigh')

from  brigh
5
#oeeee!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
from  brighe
6
#oeeeee!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
from  brighee
7
#oeeeeee!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
from  brigheee
8
#oeeeeeee!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
from  brigheeee
9
#oeeeeeeee!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
from  brigheeeee
10
#oeeeeeeeee!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
from  brigheeeeee
11
#oeeeeeeeeee!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
from  brigheeeeeee
12
#oeeeeeeeeees!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
from  brigheeeeeees
13
#oeeeeeeeeeese!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
from  brigheeeeeeese
14
#oeeeeeeeeeesee!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
from  brigheeeeeeesee
15
#oeeeeeeeeeesees!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
from  brigheeeeeeesees
16
#oeeeeeeeeeeseese!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
from  brigheeeeeeeseese
17
#oeeeeeeeeeeseeses!!!!!!!!!!!!!!!!!!!!!!!!!!!!
from  brigheeeeeeeseeses
18
#oeeeeeeeeeeseesese!!!!!!!!!!!!!!!!!!!!!!!!!!!
from  brigheeeeeeeseesese
19
#oeeeeeeeeeeseese

'brigheeeeeeeseesesesesesesesesesesesesesesese'